In [1]:
import pickle
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import struct

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
from open3d.web_visualizer import draw

[Open3D INFO] Resetting default logger to print to terminal.


In [16]:
def save_pc(pcd):
    img_width, img_height = (1920, 1080)

    renderer_pc = o3d.visualization.rendering.OffscreenRenderer(img_width, img_height)
    renderer_pc.scene.set_background(np.array([0, 0, 0, 0]))
    renderer_pc.scene.add_geometry("pcd", pcd)

    # Optionally set the camera field of view (to zoom in a bit)
    vertical_field_of_view = 15.0  # between 5 and 90 degrees
    aspect_ratio = img_width / img_height  # azimuth over elevation
    near_plane = 0.1
    far_plane = 50.0
    fov_type = o3d.visualization.rendering.Camera.FovType.Vertical
    renderer_pc.scene.camera.set_projection(vertical_field_of_view, aspect_ratio, near_plane, far_plane, fov_type)

    # Look at the origin from the front (along the -Z direction, into the screen), with Y as Up.
    center = [0, 0, 0]  # look_at target
    eye = [0, 0, 2]  # camera position
    up = [0, 1, 0]  # camera orientation
    renderer_pc.scene.camera.look_at(center, eye, up)

    depth_image = np.asarray(renderer_pc.render_to_image())
    np.save('depth', depth_image)

    normalized_image = (depth_image - depth_image.min()) / (depth_image.max() - depth_image.min())
    plt.imshow(normalized_image)
    plt.savefig('depth.png')

In [4]:
def convert_kitti_bin_to_pcd(binFilePath):
    size_float = 4
    list_pcd = []
    with open(binFilePath, "rb") as f:
        byte = f.read(size_float * 4)
        while byte:
            x, y, z, intensity = struct.unpack("ffff", byte)
            list_pcd.append([x, y, z])
            byte = f.read(size_float * 4)
    np_pcd = np.asarray(list_pcd)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(np_pcd)
    return pcd

In [5]:
with open("waymo/seq_0_frame_0.pkl", "rb") as f:
    data = pickle.load(f)

In [6]:
data["lidars"]['points_xyz'].shape, data["lidars"]['points_feature'].shape

((178624, 3), (178624, 2))

In [7]:
data["lidars"]['points_xyz']

array([[-66.24609   ,  11.540039  ,   5.059372  ],
       [-66.20703   ,  11.697266  ,   5.0584106 ],
       [-66.19629   ,  11.859375  ,   5.05867   ],
       ...,
       [ -5.4152775 ,   9.216788  ,  -0.5199143 ],
       [ -5.0214157 ,   9.377574  ,  -0.511406  ],
       [ -1.3446324 ,  -5.454536  ,  -0.08668792]], dtype=float32)

In [8]:
data["lidars"]['points_feature'].min(), data["lidars"]['points_feature'].max()

(0.0, 41728.0)

In [9]:
data["lidars"]['points_feature']

array([[0.1875    , 0.01171064],
       [0.34179688, 0.01171064],
       [0.1875    , 0.05855322],
       ...,
       [0.0300293 , 0.        ],
       [0.04711914, 0.        ],
       [0.02087402, 0.80217904]], dtype=float32)

In [10]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(data["lidars"]['points_xyz'])

In [ ]:
save_pc(pcd)

In [ ]:
pcd = convert_kitti_bin_to_pcd("000000.bin")

In [11]:
pcd_down = pcd.voxel_down_sample(voxel_size=1)


In [12]:
o3d.visualization.draw_geometries([pcd])

libGL error: No matching fbConfigs or visuals found
libGL error: failed to load driver: swrast


[Open3D WARNING] GLFW Error: GLX: Forward compatibility requested but GLX_ARB_create_context_profile is unavailable
[Open3D WARNING] Failed to create window
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [14]:
!jupyter notebook list

Currently running servers:


## Ground removal

In [24]:
plane_cloud, inliers = pcd.segment_plane(distance_threshold=0.3, ransac_n=10, num_iterations=500)
ground_cloud = pcd.select_by_index(inliers)
cloud_ground_removed = pcd.select_by_index(inliers, invert=True)
ground_cloud.paint_uniform_color([0.6, 0.6, 0.6])

PointCloud with 66510 points.

In [39]:
o3d.visualization.draw_geometries([cloud_ground_removed, ground_cloud])

In [25]:
_, inlier_indices = cloud_ground_removed.remove_statistical_outlier(100, 2)
outlier_cloud = cloud_ground_removed.select_by_index(inlier_indices, invert=True)
inlier_cloud = cloud_ground_removed.select_by_index(inlier_indices)
outlier_cloud.paint_uniform_color([0.1, 0, 0])
inlier_cloud.paint_uniform_color([0.6, 0.6, 0.6])

PointCloud with 57308 points.

Remove statistical outliers: [========================================] 100%


In [21]:
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

## DBSCAN clustering

In [26]:
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(
        inlier_cloud.cluster_dbscan(eps=0.75, min_points=100, print_progress=True))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
inlier_cloud.colors = o3d.utility.Vector3dVector(colors[:, :3])

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.Precompute neighbors.[=>                                      ] 2%
[Open3D DEBUG] Compute Clusters
Precompute neighbors.[========[Open3D DEBUG] Done Compute Clusters: 23
Precompute neighbors.[========================================] 100%
Clusteringpoint cloud has 23 clusters              ] 15%


In [28]:
o3d.visualization.draw_geometries([inlier_cloud])

In [10]:
z_idx = int(np.argmin(np.var(pcd.points, axis = 0)))
print(z_idx)

2
